### Imports


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from joblib import Parallel, delayed # Run functions in parallel
# import scipy as sc # 
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import datetime
from enum import Enum


c:\users\george\anaconda3\envs\tensor_cuda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\george\anaconda3\envs\tensor_cuda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\users\george\anaconda3\envs\tensor_cuda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Global variables

In [2]:
data_dir = '../dataset/optiver-realized-volatility-prediction/'

use_GPU = True

warnings.filterwarnings('ignore')
# pd.set_option('max_columns', 300)


### Functional Variables

In [3]:
buckets_count = 9
max_seconds_in_bucket = 600  # range 0-600 (seconds) for each observation

buckets = np.linspace(0, max_seconds_in_bucket, buckets_count, dtype=int) #  List used for binning (used as a name:lower bound of group) 
buckets = buckets[:-1]

output_dir = 'run_results/'
test_id = '2nd_attempt' + str(buckets_count)
test_dir = output_dir+test_id+"/"

# Number of columns that should have a value (not nan)
nan_threshold = 15

### Helper Functions

In [4]:
# wap of 1st entry in book
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1']+ df['ask_size1'])
    return wap

# wap of 2nd entry in book
def calc_wap2(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def unique_count(series):
    return len(np.unique(series))

def read_main_csvs():
    train = pd.read_csv(data_dir + 'train.csv')
    test = pd.read_csv(data_dir + 'test.csv')
    # Create row_id to merge data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'The training set has {train.shape[0]} rows')
    return train, test

def calc_model_importance(model, feature_names=None, importance_type='gain'):
    importance_df = pd.DataFrame(model.feature_importance(importance_type=importance_type),
                                 index=feature_names,
                                 columns=['importance']).sort_values('importance')
    return importance_df

def calc_mean_importance(importance_df_list):
    mean_importance = np.mean(
        np.array([df['importance'].values for df in importance_df_list]), axis=0)
    mean_df = importance_df_list[0].copy()
    mean_df['importance'] = mean_importance
    return mean_df

def plot_importance(importance_fig, filename):
    if not os.path.exists(os.path.dirname(test_dir)):
        try:
            os.makedirs(os.path.dirname(test_dir))
        except:
            print("Could not create dir",filename)
    timestamp = datetime.datetime.now().strftime('%m-%d-%H_%M')
    importance_fig.figure.savefig(test_dir+filename+timestamp+"plot"+".png")
    
    
def save_results_to_file(filename,params,rmspe_score):
    if not os.path.exists(os.path.dirname(test_dir)):
        try:
            os.makedirs(os.path.dirname(test_dir))
        except:
            print("Could not create dir")
    timestamp = datetime.datetime.now().strftime('%m-%d-%H_%M')
    with open(test_dir+filename+timestamp+'.txt', 'w') as f:
        f.write(f'The test RMSPE is {rmspe_score} \n Hyperparams: {params}')


# Calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Early stoping, root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

# Create features for order book data (for a given stock file)
def create_bucketed_features(dataframe, aggregate_features_map):
    features = pd.DataFrame()

    for bucket_start_time in buckets:
        # Group into time buckets
        bucketed_features = dataframe[dataframe['seconds_in_bucket']
                                      .between(bucket_start_time, bucket_start_time + buckets[1] - buckets[0])].groupby(['time_id']).agg(aggregate_features_map)
        bucketed_features.columns = ['_'.join(col) for col in bucketed_features.columns]
        bucketed_features = bucketed_features.add_suffix('_' + str(bucket_start_time))
        if bucket_start_time == 0:
            features = bucketed_features
        else:
            features = features.merge(bucketed_features, how = 'outer', left_on = 'time_id', right_on = 'time_id')
    
    return features


### 2nd Attempt feature engineering

In [5]:
def group_order_book_in_time_buckets(file_path):
    dataframe = pd.read_parquet(file_path)
    
    dataframe['wap2'] = calc_wap1(dataframe)
    dataframe['price_diff'] = abs(dataframe['ask_size2'] - dataframe['bid_price2'])
  
    aggregate_features_map = {
        'wap2': [np.mean],
        'price_diff': [np.mean],
        
        'bid_price1': [np.mean],
        'ask_price1': [np.mean],
        
        'bid_price2': [np.mean],
        'ask_price2': [np.mean],
        
        'bid_size1': [np.mean],
        'ask_size1': [np.mean],
        
        'bid_size2': [np.mean],
        'ask_size2': [np.mean],
    }
    
        
    features = create_bucketed_features(dataframe,aggregate_features_map)
    features = features.reset_index(level=0)
    # Create row_id to merge
    stock_id = file_path.split('=')[1]
    features['row_id'] = features['time_id'].apply(lambda x: f'{stock_id}-{x}')
    features.drop(['time_id'], axis = 1, inplace = True)
    return features

# Split into time buckets within a time_id
def group_trade_book_in_time_buckets(file_path):
    dataframe = pd.read_parquet(file_path)
    
    aggregate_features_map = {
        'price':[np.mean, np.max, np.min],
        'size':[np.mean, np.max, np.min],
        'order_count':[np.mean, np.max, np.min],
    }


    features = create_bucketed_features(dataframe,aggregate_features_map)
    features = features.reset_index(level=0)
    features = features.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    features['row_id'] = features['trade_time_id'].apply(lambda x:f'{stock_id}-{x}')
    features.drop(['trade_time_id'], axis = 1, inplace = True)
    return features

# Create group stats for all stocks in each time_id
def get_group_time_id_stats(dataframe):
    group_stats = []

    for bucket in buckets:
        group_stats.append(f'wap2_mean_{bucket}')
        group_stats.append(f'price_diff_mean_{bucket}')



    time_id = dataframe.groupby(['time_id'])[group_stats].agg([np.mean])
    time_id.columns = ['_'.join(col) for col in time_id.columns]
    time_id = time_id.add_suffix('_' + 'time_id_groupstat')
    
    # Merge result with main dataframe
    dataframe = dataframe.merge(time_id, how = 'inner', left_on = ['time_id'], right_on = ['time_id'])
    return dataframe



### Main Control

In [6]:
def load_stock(stock_id,is_training_set):
    # Train
    if is_training_set:
        order_book_path = f"{data_dir}book_train.parquet/stock_id={str(stock_id)}"
        trade_book_path = f"{data_dir}trade_train.parquet/stock_id={str(stock_id)}"
    # Test
    else:
        order_book_path = f"{data_dir}book_test.parquet/stock_id={str(stock_id)}"
        trade_book_path = f"{data_dir}trade_test.parquet/stock_id={str(stock_id)}"
    _tmp = None
    # Create order and trade book features and merge them

    _tmp = pd.merge(group_order_book_in_time_buckets(order_book_path), group_trade_book_in_time_buckets(trade_book_path), on = ['row_id'], how = 'left')
    return _tmp
    

In [7]:
%%time

train, test = read_main_csvs()

train_stock_ids = train['stock_id'].unique()
dataframes = None
# Features engineering & binning for training set
dataframes = Parallel(n_jobs = -1, verbose = 1)(delayed(load_stock)(stock_id,is_training_set=True) for stock_id in train_stock_ids)
# for stock_id in train_stock_ids:
#     dataframes = load_stock(stock_id, is_training_set=True)
train_ = pd.concat(dataframes, ignore_index = True)

train = train.merge(train_, on = ['row_id'], how = 'left')


test_stock_ids = test['stock_id'].unique()
print(test_stock_ids)
dataframes = None
# Features engineering & binning for test set
dataframes = Parallel(n_jobs = -1, verbose = 1)(delayed(load_stock)(stock_id,is_training_set=False) for stock_id in test_stock_ids)
# for stock_id in test_stock_ids:
#     dataframes = load_stock(stock_id, is_training_set=False)
test_ = pd.concat(dataframes, ignore_index = True)

test = test.merge(test_, on = ['row_id'], how = 'left')

# Generate group stats for time_id and stock_id
 
train = get_group_time_id_stats(train)
test = get_group_time_id_stats(test)


The training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:   16.6s finished


[0]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


CPU times: total: 10.2 s
Wall time: 25.3 s


In [8]:
gain_importance_list = []
split_importance_list = []



### Sanity Checks

In [9]:
len(train.index)

428932

In [10]:
train.head()

,stock_id,time_id,target,row_id,wap2_mean_0,price_diff_mean_0,bid_price1_mean_0,ask_price1_mean_0,bid_price2_mean_0,ask_price2_mean_0,...,wap2_mean_225_mean_time_id_groupstat,price_diff_mean_225_mean_time_id_groupstat,wap2_mean_300_mean_time_id_groupstat,price_diff_mean_300_mean_time_id_groupstat,wap2_mean_375_mean_time_id_groupstat,price_diff_mean_375_mean_time_id_groupstat,wap2_mean_450_mean_time_id_groupstat,price_diff_mean_450_mean_time_id_groupstat,wap2_mean_525_mean_time_id_groupstat,price_diff_mean_525_mean_time_id_groupstat
0,0,5,0.004136,0-5,1.002237,118.515404,1.001954,1.002829,1.001838,1.002981,...,1.002719,581.660465,1.002687,589.312578,1.002546,549.179006,1.002421,580.797176,1.002107,601.708474
1,1,5,0.006340,1-5,1.002095,63.012460,1.001696,1.002398,1.001590,1.002516,...,1.002719,581.660465,1.002687,589.312578,1.002546,549.179006,1.002421,580.797176,1.002107,601.708474
2,2,5,0.001848,2-5,1.000654,209.903615,1.000605,1.000736,1.000523,1.000817,...,1.002719,581.660465,1.002687,589.312578,1.002546,549.179006,1.002421,580.797176,1.002107,601.708474
3,3,5,0.005300,3-5,0.999835,197.162303,0.999578,1.000168,0.999462,1.000281,...,1.002719,581.660465,1.002687,589.312578,1.002546,549.179006,1.002421,580.797176,1.002107,601.708474
4,4,5,0.004468,4-5,1.002056,93.642818,1.001596,1.002420,1.001336,1.002485,...,1.002719,581.660465,1.002687,589.312578,1.002546,549.179006,1.002421,580.797176,1.002107,601.708474


In [11]:
print(len(train) - len(train.dropna(thresh=nan_threshold, axis=0)))
print(len(test) - len(test.dropna()))

# drop rows (axis=0) if more than 3 columns are nan
train.dropna(inplace=True, thresh=nan_threshold, axis=0)


0
3


In [12]:

test.head()

,stock_id,time_id,row_id,wap2_mean_0,price_diff_mean_0,bid_price1_mean_0,ask_price1_mean_0,bid_price2_mean_0,ask_price2_mean_0,bid_size1_mean_0,...,wap2_mean_225_mean_time_id_groupstat,price_diff_mean_225_mean_time_id_groupstat,wap2_mean_300_mean_time_id_groupstat,price_diff_mean_300_mean_time_id_groupstat,wap2_mean_375_mean_time_id_groupstat,price_diff_mean_375_mean_time_id_groupstat,wap2_mean_450_mean_time_id_groupstat,price_diff_mean_450_mean_time_id_groupstat,wap2_mean_525_mean_time_id_groupstat,price_diff_mean_525_mean_time_id_groupstat
0,0,4,0-4,1.000405,18.667011,1.000049,1.000606,0.999656,1.000721,157.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print(len(train.index))
train.columns

428932


Index(['stock_id', 'time_id', 'target', 'row_id', 'wap2_mean_0',
       'price_diff_mean_0', 'bid_price1_mean_0', 'ask_price1_mean_0',
       'bid_price2_mean_0', 'ask_price2_mean_0',
       ...
       'wap2_mean_225_mean_time_id_groupstat',
       'price_diff_mean_225_mean_time_id_groupstat',
       'wap2_mean_300_mean_time_id_groupstat',
       'price_diff_mean_300_mean_time_id_groupstat',
       'wap2_mean_375_mean_time_id_groupstat',
       'price_diff_mean_375_mean_time_id_groupstat',
       'wap2_mean_450_mean_time_id_groupstat',
       'price_diff_mean_450_mean_time_id_groupstat',
       'wap2_mean_525_mean_time_id_groupstat',
       'price_diff_mean_525_mean_time_id_groupstat'],
      dtype='object', length=172)

In [14]:
train = train.drop(['time_id'], axis = 1)
test = test.drop(['time_id'], axis = 1)

### Model Training

In [15]:
def train_and_evaluate(train, test):
    # Hyperparammeters (just basic)
    hyperparameters = {
        'objective': 'rmse',  
        'boosting_type': 'gbdt',
        #       'boosting_type': 'dart', # Try to solve over-specialization problem in gbdt
        #       'boosting_type': 'goss', # Goss provides a new sampling method for GBDT by separating those instances with larger gradients.
        'num_iterations': 5678, # Num_iterations specifies the number of boosting iterations (trees to build). The more trees you build the more accurate your model can be at the cost of:
        'n_estimators': 20000,#controls the number of decision trees 
        'min_data_in_leaf': 9,
        'max_bin': 75, # If you define max_bin 255 that means we can have a maximum of 255 unique values per feature. Then Small max_bin causes faster speed and large value improves accuracy.
        'num_leaves': 27,
        'max_depth': 9, # This parameter control max depth of each trained tree and will have impact on:       The best value for the num_leaves parameter     Model Performance     Training Time  Pay attention If you use a large value of max_depth, your model will likely be over fit  to the train set
        'n_jobs': -1,
        'learning_rate': 0.095, # learning_rate > 0.0Typical: 0.05.
        'feature_fraction': 0.9, # or sub_feature deals with column sampling, LightGBM will randomly select a subset of features on each iteration (tree)
        'bagging_fraction': 0.9, # you can specify the percentage of rows used per tree building iteration.
        'verbose': -1,
        # 'is_unbalance': True #One of the problems you may face in the binary classification problems is how to deal with the unbalanced datasets. Obviously, you need to balance positive/negative samples but how exactly can you do that in lightgbm      
    }
    
    if use_GPU:
        hyperparameters.update(
            {'device' : 'gpu',
            'gpu_platform_id' : 0,
            'gpu_device_id' : 0,}
        )
     
   
    # We need to drop row_id as it contains info from both time_id and stock_id and time id should be out of the features | should only be used in predictions
    x = train.drop(['row_id', 'target'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id'], axis = 1)

    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    
    test_predictions = np.zeros(x_test.shape[0])
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])

    
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = hyperparameters, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, 
                          early_stopping_rounds = 50, 
                          verbose_eval = 50,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
    
    importance_plot = lgb.plot_importance(model,max_num_features=7,figsize=(32,15))
    plot_importance(importance_plot, "feature_importance_plot")
    rmspe_score = rmspe(y, oof_predictions)
    print(f'The out of folds RMSPE is {rmspe_score}')

    return test_predictions

In [16]:
%%time

# Traing and evaluating
test_predictions = train_and_evaluate(train, test)

Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000629493	training's RMSPE: 0.291238	valid_1's rmse: 0.000635981	valid_1's RMSPE: 0.294741
[100]	training's rmse: 0.000578685	training's RMSPE: 0.267732	valid_1's rmse: 0.000591488	valid_1's RMSPE: 0.274121
[150]	training's rmse: 0.000550692	training's RMSPE: 0.25478	valid_1's rmse: 0.000568727	valid_1's RMSPE: 0.263573
[200]	training's rmse: 0.00053082	training's RMSPE: 0.245586	valid_1's rmse: 0.000553537	valid_1's RMSPE: 0.256533
[250]	training's rmse: 0.00051526	training's RMSPE: 0.238388	valid_1's rmse: 0.000543893	valid_1's RMSPE: 0.252064
[300]	training's rmse: 0.000502706	training's RMSPE: 0.23258	valid_1's rmse: 0.000536489	valid_1's RMSPE: 0.248632
[350]	training's rmse: 0.00049089	training's RMSPE: 0.227113	valid_1's rmse: 0.000528851	valid_1's RMSPE: 0.245093
[400]	training's rmse: 0.00048149	training's RMSPE: 0.222764	valid_1's rmse: 0.000523815	valid_1's RMSPE: 0.242758
[4

[1150]	training's rmse: 0.000406239	training's RMSPE: 0.188024	valid_1's rmse: 0.000500502	valid_1's RMSPE: 0.23158
[1200]	training's rmse: 0.000403416	training's RMSPE: 0.186718	valid_1's rmse: 0.000500169	valid_1's RMSPE: 0.231426
Early stopping, best iteration is:
[1174]	training's rmse: 0.000404752	training's RMSPE: 0.187336	valid_1's rmse: 0.00049996	valid_1's RMSPE: 0.231329
Training fold 4



KeyboardInterrupt



In [17]:
# Save test predictions for submission
test['target'] = test_predictions
test[['row_id', 'target']].to_csv('submission.csv',index = False)

NameError: name 'test_predictions' is not defined